# Методы HIVES и DHF для группового принятия решений

Этот ноутбук демонстрирует использование методов **HIVES** (Hierarchical Voting-based Evaluation System) и **DHF** (Dual Hesitant Fuzzy) консенсуса.

Репозиторий: https://github.com/Karperash/Metod-HIVES-and-DHF


## 1. Установка зависимостей и клонирование репозитория


In [ ]:
# Клонируем репозиторий
!git clone https://github.com/Karperash/Metod-HIVES-and-DHF.git
%cd Metod-HIVES-and-DHF


In [ ]:
# Устанавливаем зависимости
!pip install -q numpy


## 2. Пример 1: Запуск только HIVES


In [ ]:
# Запускаем метод HIVES на примере данных
!python main.py hives examples/hives/input.json


## 3. Пример 2: Комбинированный метод (DHF → HIVES)

**Важно:** Сначала запустим без rotation (замены эксперта), затем покажем пример с rotation.


In [ ]:
# Запускаем комбинированный метод БЕЗ rotation (замены эксперта)
# Сначала DHF оптимизирует веса экспертов, затем HIVES использует их для ранжирования
!python main.py combined examples/combined/combined_input_program_no_rotation.json


### 3.1. Пример с rotation (замена эксперта)

Теперь, когда у нас есть результат предыдущего запуска, можно использовать rotation:


In [ ]:
# Сначала убедимся, что папка outputs существует и есть результат
import os
os.makedirs('outputs', exist_ok=True)

# Если файл еще не создан, создадим его, запустив без rotation
if not os.path.exists('outputs/combined_result_program.json'):
    print("Создаем базовый результат...")
    !python main.py combined examples/combined/combined_input_program_no_rotation.json

# Теперь можно запустить с rotation
print("\nЗапускаем с rotation (замена эксперта):")
!python main.py combined examples/combined/combined_input_program.json


## 4. Пример 3: Эксперимент - сравнение весов экспертов


In [ ]:
# Сравниваем веса экспертов в разных сценариях
# Используем файл без rotation для эксперимента
!python main.py experiment examples/combined/combined_input_program_no_rotation.json


## 5. Пример 4: HIVES → compat3 → DHF


In [ ]:
# Сначала HIVES, затем проверка консенсуса, затем оптимизация DHF
!python main.py hives-compat-dhf examples/combined/combined_input_program_no_rotation.json


## 6. Использование Python API напрямую


In [ ]:
import sys
sys.path.insert(0, '/content/Metod-HIVES-and-DHF')

from hives_dhf.json_input import load_decision_problem_from_json
from hives_dhf.hives_method import hives_rank
import numpy as np

# Загружаем данные
problem = load_decision_problem_from_json('examples/hives/input.json')

# Получаем матрицы
A = problem.aggregated_performance()  # Матрица оценок альтернатив
W = problem.weights_matrix()          # Матрица весов критериев у экспертов

expert_ids = [e.id for e in problem.experts] if problem.experts else [f"DM{i+1}" for i in range(W.shape[0])]

# Выполняем ранжирование
result = hives_rank(A=A, W=W, expert_ids=expert_ids)

print("Результаты HIVES:")
print(f"Веса критериев (gamma_scaled): {np.round(result['gamma_scaled'], 2)}")
print(f"\nОценки альтернатив:")
for i, alt in enumerate(problem.alternatives):
    print(f"  {alt}: {result['alt_scores'][i]:.2f}")
print(f"\nРанжирование: {result['ranking'] + 1}")
